# Language Models

One of the important and almost the backbone of different NLP related tasks! 
Let's assume we want to calculate the probability of a sentence  $P(S) = P(W_1 , … , W_m)$ or if we want to predict the next word given the probabilities of previous words $P(W_n|W_1 , … , W_m)$ then Language Model is the task which does it. Now, while calculating probability of next word given previous words usually we have to look back so far (in ideal situation, all the previous words). Thankfully, we use `Markov assumption` and only consider few words from the past. 
So, in this notebook we will first develop statistical and then Deep Learning approaches.

Let's starts statistical methods. I mentioned about consideration of previous words, this is where the concept of `n-grams` kicks in. N in `n-grams` specifies number of words we want to look back for the prediction of next words! So how does this `n-grams` works?
Suppose we want to generate a sentence from one single word `The`! then first step would be to find the most probable word after the word `The` and to find this most probable word we have to calculate probalities of all the words from the corpus given the word `The`. Trust me, it's not difficult as it sounds! 
Let's consider an example. Suppose we want to calculate the probability of word `company` occuring after the given word `The` $P(Company|The)$. So, we just need to calculate how many times `The company` occured in our corpus and then just normalized this count with number of times word `The` occured in our courpus. Hence, complete forumlar would be

$$P(Company|The) = \frac{count(The, Company)}{count(The)}$$

We will use same formula for all the words in our corpus and our next predicted word would be simply the word with highest probability. Hence, the more general form of above given formula can be written as 

$$P(W_i|W_{i-1}) = \frac{count(W_{i-1}, W_i)}{count(W_{i-1})}$$

When we consider single word from the past to predict next word is known as `bi-grams` approach for language modelling as we are considering two `(bi)` words. One thing to remember is that the more words we consider from the past the more better results we will get (you will see yourself in this notebook). So if you want to consider three words from the past we call them `tri-grams`. For `tri-grams` our forumula would simply include one more word.

$$P(c|a,b) = \frac{count(a, b, c)}{count(a,b)}$$

But there is one problem, if you want to use more previous words from the past which is `Sparsity` !. For example, from last formula what if we simply cant count $count(a, b, c)$ or $count(a, b)$ because these words never occured together! Then we have two solutions:

1. Smoothing 
2. Backoff (if tri-gram is not avialable then use bi-gram if not then uni-gram)

I would simply ignore these problems in this notebook 😬. However, at the end of this notebook I have provided helpful resources to study more about them. Another thing is, for every Artificial Intelligence problem or task we must evaluate our models on seperate validation and test sets. Here I am completely ignoring this as this notebook is more about understanding and implementation of LMs. Although, if you are going to use LMs in real scenarios you must use validation set.

### Statistical method

Enough talk let's start coding. We will use `nltk` as it simplfies creation of `bi-grams` and `tri-grams`.

In [2]:
# Thanks https://nlpforhackers.io/language-models/
import random
import math
from collections import Counter, defaultdict

import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams

In [ ]:
nltk.download('punkt')
nltk.download('reuters')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maqboolkhan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

However, before coding `bi-grams` there is another appproach known as `uni-gram`. In `uni-gram` we don't consider any previous word. Hence, we only count occurance of each word and then simply normalized (divide) this count with total no. of words in the corpus. This approach also known as `bag-of-words`. 

In [13]:
uni_grams = Counter(reuters.words())  # Counting occurance of each word in our corpus
total_count = len(reuters.words())

# Compute the probabilities (uni-grams)
for word in uni_grams:
    uni_grams[word] /= float(total_count)

Now, as we have calculated `uni-grams`. Let's just check most occured tokens in our corpus.

In [14]:
uni_grams_counter = Counter(uni_grams)
uni_grams_counter.most_common(10)

[('.', 0.055021758950689205),
 (',', 0.042047741270415905),
 ('the', 0.033849129031826936),
 ('of', 0.02090707135390124),
 ('to', 0.01977743054365126),
 ('in', 0.015386126221089999),
 ('said', 0.014657438167564549),
 ('and', 0.014552260705293332),
 ('a', 0.013650988639090802),
 ('mln', 0.010481137497159917)]

Now, here comes most excited part!! lets generate language. `Uni-grams` do not consider any previous word so now it's completely upto us how we pick next word. Here, what I did is to first get random number (float) and then iterate over all the words and sum their probabilities and as soon as this sum is greater than that random number we pick the word.

In [15]:
def generate_text(grams, n, context, length):
    text = list(context)
    context = context[0] if n == 2 else context  # bigrams had , in tne context tuple hence to remedy that comma!

    for i in range(length):
        sum = 0

        r = random.random()  # For diversity in text generation

        if context:
            candidates = grams[context]
        else:
            candidates = grams

        for k in candidates.keys():
            sum += candidates[k]
            if sum > r:
                text.append(k)

                if context:
                    context = (k) if n == 2 else (context[2-n], k)

                break
    text = ['None' if token == None else token  for token in text] # Replacing None with 'None'
    return ' '.join(text)

Now, lets generate some text using our `uni-grams` model

In [653]:
generate_text(uni_grams, 1, (), 10)

', IN Allegis banks 348 said April intervention total year'

hmm, not very impressive. Let's move on to `bi-grams`.

In [227]:
def calc_probs(grams):
    # Let's transform the counts to probabilities
    for context in grams:
        context_count = float(sum(grams[context].values()))
        for next_word in grams[context]:
            grams[context][next_word] /= context_count

In [228]:
# Counting all bi-grams
bi_grams = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in reuters.sents():
    for w1, w2 in bigrams(sentence, pad_left=True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>"):
        bi_grams[(w1)][w2] += 1

# Calculating probabilities
calc_probs(bi_grams)

In [231]:
generate_text(bi_grams, 2, ('The',), 10)

'The Federal Savings System Inc and Japanese intervention ," he added'

Notice here we are passing previous word (context) and it's single word as we are using `bi-grams`.  Also notice that the generated sentence is coherent.
Now, let's try with `tri-grams`.

In [239]:
tri_grams = defaultdict(lambda: defaultdict(lambda: 0))

for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_left=True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>"):
        tri_grams[(w1, w2)][w3] += 1

calc_probs(tri_grams)

In [242]:
generate_text(tri_grams, 3, ('<s>', 'The'), 10)

'<s> The bonus award was made during the morning after analyst Daniel'

### Perplexity

Now, we have understood language models and how to generate text with them. However, one important aspect of any ML model is how to evaluate them. For this task, we can ask does our language model prefer good sentences to bad ones? does our model assign a higher probability to real or frequently observed word than ungrammatical or rarely observed one? Other than that we also have automatic measure which is `Perplexity`. It's inverse probability of our corpus (or sentence we can also calculate it for sentence) normalized by no. of words in our corpus. 

$$\sqrt[N]{ \frac{1}{P(w_1 ... w_n)}}$$

Notice here we have to calculate $P(w_1 ... w_n)$ which involve _Chain Rule of Probability_ e.g  $P(w_1 ... w_n) = P(w_1) x P(w_2|w_1) x P(w_3|w_1, w_2) ... $ and notice if one of these probabilities is zero then the whole $P(w_1 ... w_n)$ will become zero. To avoid this underflow, I will use alternate representation (formula) of perplexity involving _logs_.

$$2^{-\frac{1}{N} \sum_{i=1}^{N} log_2 P(w_i) }$$

As perplexity is inverse measure. Hence, lower Perplexity indicates better language model.

In [223]:
# https://stats.stackexchange.com/a/143638/291743
N = 0
summation = 0
for word in nltk.tokenize.wordpunct_tokenize(reuters.raw()):
    N += 1
    summation += math.log(uni_grams[word], 2)

print("Uni gram perplexity: ", pow(2, -summation * (1/N)))

Uni gram perplexity:  1077.8271341207844


In [249]:
# https://towardsdatascience.com/perplexity-in-language-models-87a196019a94
N = 0
summation = 0
for sentence in reuters.sents():
    N += len(sentence) + 2 # +2 for <s> and </s> 
    for w1, w2 in bigrams(sentence, pad_left=True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>"): 
        summation += math.log(bi_grams[(w1)][w2], 2)
print("Bi gram perplexity: ", pow(2, -summation * (1/N)))

Bi gram perplexity:  40.65095064037762


In [254]:
N = 0
summation = 0
for sentence in reuters.sents():
    N += len(sentence) + 4 # +4 for <s> and </s> 
    for w1, w2, w3 in trigrams(sentence, pad_left=True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>"): 
        summation += math.log(tri_grams[(w1, w2)][w3], 2)
print("Tri gram perplexity: ", pow(2, -summation * (1/N)))

Tri gram perplexity:  5.9633012418563265


So as I said earlier, that the more words we consider from the past the more better results we will get and one can see it here. 
Now, it's time to do the same task of Language Modeling with Deep Learning.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from torchtext.vocab import build_vocab_from_iterator
import torch.nn.functional as F

from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
hyp_params = {
    "batch_size": 32,
    "embedding_dim": 125,
    "hidden_dim": 2,
    "sequence_len": 10
}

We will use simple vanilla LSTM model. One important thing to understand here is how our input to LSTM model and output will look like. We will use _window mechanism_ which is simillar as `n-gram` approach. Let's consider an example, suppose we have a sentence "The quick brown fox jumps over the lazy dog" and we set the size of input and output _sentence window_ is `3` hence our first _input_ would be "The quick brown" and output would be "quick brown fox". Following the same approach, the second input would be "quick brown fox" and output would be "brown fox jumps". The next `LMDataset` class implementd this logic and it takes `sentence_window` as its input. If you look _`__getitem__`_ method of `LMDataset` class uses this `sentence_window` and _`slider`_ to create our input and output as I explained.

In [13]:
class LMDataset(Dataset):
    def __init__(self, nltk_corpus, sentence_window = 50, train_vocab=None):
        self.corpus = nltk.tokenize.wordpunct_tokenize(nltk_corpus)
        self.vocab = train_vocab if train_vocab else self._build_vocab()
        self.sentence_window = sentence_window

        self.slider = -1

    def __len__(self):
        return math.floor(len(self.corpus)/self.sentence_window)

    def __getitem__(self, item):
        self.slider += 1

        src_text_tokens = self.corpus[self.slider * self.sentence_window : (self.slider + 1) * self.sentence_window]
        trg_text_tokens = self.corpus[(self.slider * self.sentence_window) + 1 : ((self.slider + 1) * self.sentence_window) + 1]
        
        return {
            'src': self.vocab.lookup_indices(src_text_tokens),
            'trg': self.vocab.lookup_indices(trg_text_tokens)
        }

    def _build_vocab(self):
        vocab = build_vocab_from_iterator([self.corpus], specials=["<unk>","<pad>"])
        vocab.set_default_index(vocab['<unk>'])

        return vocab

In [14]:
def collate_fn(batch, pad_value, device):
    trgs = []
    srcs = []
    for row in batch:
        srcs.append(torch.tensor(row["src"], dtype=torch.long).to(device))
        trgs.append(torch.tensor(row["trg"]).to(device))

    padded_srcs = pad_sequence(srcs, padding_value=pad_value)
    padded_trgs = pad_sequence(trgs, padding_value=pad_value)
    return {"src": padded_srcs, "trg": padded_trgs}


train_lmds = LMDataset(reuters.raw(), hyp_params["sequence_len"])

pad_value = train_lmds.vocab['<pad>']

train_dt = DataLoader(train_lmds, 
                      batch_size=hyp_params["batch_size"],
                      shuffle=True,
                      collate_fn=lambda batch_size: collate_fn(batch_size, pad_value, device))

In [15]:
class LM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()

        # Embedding is just an lookup table of size "vocab_size"
        # and each element has "embedding_size" dimension
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden_state=None, cell_state=None):
        # Shape --> [Sequence_length , batch_size , embedding dims]
        embedding = self.embedding(x)
        # Shape --> (output) [Sequence_length , batch_size , hidden_size]
        # Shape --> (hs, cs) [num_layers, batch_size size, hidden_size]
        if hidden_state is not None:
            outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
        else:
            outputs, (hidden_state, cell_state) = self.LSTM(embedding)

        '''
            Unlike Classification task, 
            here we are making use of outputs from our LSTM.
        '''
        # shape --> (linear_outputs) (10, 32, 41602) sentence len, batch size, vocab size
        linear_outputs = self.fc(outputs)

        return linear_outputs, hidden_state, cell_state

In [16]:
model = LM(len(train_lmds.vocab), hyp_params["embedding_dim"], hyp_params["hidden_dim"]).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

Let's train our deep learning model just for 10 epochs

In [17]:
for epoch in range(10):
    model.train()
    epoch_loss = 0
    print('Epoch: ', epoch)
    for idx, batch in enumerate(tqdm(train_dt)):
        src = batch["src"]  # shape --> e.g. (10, 32) sentence len, batch size
        trg = batch["trg"]  # shape --> e.g. (10, 32) sentence len, batch size

        trg = trg.view(-1) # making them linear (1d) --> bsz * seq len

        # Clear the accumulating gradients
        optimizer.zero_grad()

        # shape --> (10, 32, 41602) sentence len, batch size, vocab
        output, _, _ = model(src)

        # Calculate the loss value for every epoch
        loss = criterion(output.view(-1, len(train_lmds.vocab)), trg)

        # Calculate the gradients for weights & biases using back-propagation
        loss.backward()

        epoch_loss += loss.item()

        # Clip the gradient value is it exceeds > 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Update the weights values
        optimizer.step()
    print(f'\tTrain loss: {epoch_loss/len(train_dt)}, Train perplexity: {math.exp(epoch_loss/len(train_dt))}')
    train_lmds.slider = -1

Epoch:  0


100%|██████████| 5378/5378 [01:57<00:00, 45.75it/s]


	Train loss: 7.599785830461212, Train perplexity: 1997.7679882347986
Epoch:  1


100%|██████████| 5378/5378 [01:56<00:00, 46.07it/s]


	Train loss: 6.8328121472752725, Train perplexity: 927.7962470172789
Epoch:  2


...

	Train loss: 6.289007102569801, Train perplexity: 538.6182708026654
Epoch:  8


100%|██████████| 5378/5378 [01:58<00:00, 45.50it/s]


	Train loss: 6.245247852141727, Train perplexity: 515.556990997051
Epoch:  9


100%|██████████| 5378/5378 [01:58<00:00, 45.43it/s]

	Train loss: 6.20877730806506, Train perplexity: 497.09308785487667


So, 10 epochs were not enough even for our _`train`_ dataset as you can see we have very high perplexity infact 10x more than `bi-grams` models. However, I am sure training for more epochs will lead to better results.
But how we just calculated `perplexity` using `Cross-entropy` loss 🤔? So, it turned out there is a relationship between `perplexity` using `Cross-entropy` loss. I am not going to explain here but you can check this article
https://towardsdatascience.com/the-relationship-between-perplexity-and-entropy-in-nlp-f81888775ccc

In [18]:
torch.save(model.state_dict(),'lm.pt')

In [19]:
pre_model = torch.load('lm.pt', map_location=device)
model.load_state_dict(pre_model)
model.eval()

LM(
  (embedding): Embedding(41602, 125)
  (LSTM): LSTM(125, 2)
  (fc): Linear(in_features=2, out_features=41602, bias=True)
)

### Text generation

As Perplexity is very high so we can expect low quality generated text.

In [55]:
inp = "The"

hs = None
cs = None
for i in range(10):
    inp_ind = torch.tensor([train_lmds.vocab[inp]]).unsqueeze(1)
    with torch.no_grad():
        output, hs, cs = model(inp_ind, hs, cs)
    word_weights = output.squeeze().exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    inp = train_lmds.vocab.lookup_token(word_idx)
    print(inp, end=" ")

Increased Public support earnings , much of 17 dlrs . 

### Calculating perplexity of a single sentence

We already have calculated `perplexity` of whole corpus but what if we want to calculate it for just one sentence ?

In [57]:
# Thanks: https://github.com/flairNLP/flair/issues/498#issuecomment-465192107
sentence = reuters.sents()[0]

# this was the main step
inp = sentence[:-1]
trg = sentence[1:]

trg_tensor = torch.tensor(train_lmds.vocab.lookup_indices(trg)).to(device)

inp_tensor = torch.tensor(train_lmds.vocab.lookup_indices(inp)).unsqueeze(1)
with torch.no_grad():
    output, _, _ = model(inp_tensor)
    loss = criterion(output.view(-1, len(train_lmds.vocab)), trg_tensor).item()

math.exp(loss)

795.0305084679243

### Calculating probability of a single sentence

In [58]:
hs = None
cs = None
sentence = reuters.sents()[0]
probs = 0
for idx in range(len(sentence)):
    inp_ind = torch.tensor(train_lmds.vocab.lookup_indices(sentence[:idx+1])).unsqueeze(1)
    with torch.no_grad():
        output, _, _ = model(inp_ind)
    output = F.softmax(output.squeeze().detach())
    if idx > 0:
        # Getting the last one (so far probabilities)
        output = output[idx]
    
    probs += output[train_lmds.vocab[sentence[idx]]].item()

print(probs/len(sentence))

0.002715979320863876


D:\maq\Project-NLP\env\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
